# Crime Statistics

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


import time

# Set up the WebDriver (ensure chromedriver is in your PATH)
driver = webdriver.Chrome()

# Open a website
driver.get("https://www.police.wa.gov.au/Crime/CrimeStatistics#/start")

# # Find an input element (adjust the selector as needed)
# input_element = driver.find_element("name", "q")

# # Type something in the input field
# input_element.send_keys("example search")

# # Simulate pressing Enter
# input_element.send_keys(Keys.RETURN)

# # Wait for some time to see the result (adjust the sleep time as needed)
# time.sleep(5)

# # Close the browser
# driver.quit()

In [5]:
from selenium.webdriver.common.by import By

In [6]:
driver.find_element(By.CSS_SELECTOR, '.btn-locality-group[ng-model="location"][uib-btn-radio="\'Suburb\'"]').click()

In [ ]:
import time

In [43]:
# Find the input field by its ID
input_field = driver.find_element(By.ID, 'searchLocality_value')
input_field.send_keys('WILLETTON')
time.sleep(1)
input_field.send_keys(Keys.ENTER)
#driver.implicitly_wait(10)
#input_field.send_keys(Keys.ENTER)


# Input the suburb name
# suburb_name = 'SUBIACO'  # Replace with the desired suburb name


In [37]:
input_field = driver.find_element(By.ID, 'searchLocality_value')
input_field.send_keys(Keys.ENTER)

In [ ]:
# # Find an input element (adjust the selector as needed)
input_element = driver.find_element("name", "q")



In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
# Visit the WA Police site
url = 'https://www.police.wa.gov.au/Crime/CrimeStatistics#/start'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

False

In [4]:
# Scrape the website
html = browser.html

# Create a BeautifulSoup object from the scraped HTML
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Automate clicking the Suburb button
browser.find_by_css('.btn-locality-group[ng-model="location"][uib-btn-radio="\'Suburb\'"]').click()

In [8]:
# Find the input field by its ID
input_field = browser.find_by_id('searchLocality_value')

# Input the suburb name
suburb_name = 'SUBIACO'  # Replace with the desired suburb name
input_field.fill(suburb_name)

In [10]:
# Find an input element (adjust the selector as needed)
# input_element = browser.find_by_id("id", "searchLocality_value")

# Simulate pressing Enter
input_field.send_keys(Keys.RETURN)

AttributeError: 'ElementList' object has no attribute 'send_keys'

In [ ]:
# Simulate pressing Enter
input_element.send_keys(Keys.RETURN)

# Wait for some time to see the result (adjust the sleep time as needed)
time.sleep(5)

In [15]:
# Select one news article with select_one()
slide_elem = soup.select_one('div.list_text')

In [18]:
slide_elem = soup.find('div', class_='list_text')

In [19]:
# Scrape the title of the news article
title_elem = slide_elem.find('div', class_='content_title')
print(title_elem)

<div class="content_title">NASA's MAVEN Observes Martian Light Show Caused by Major Solar Storm</div>


In [20]:
# Extract the text of the title with get_text()
title = title_elem.get_text()
print(title)

NASA's MAVEN Observes Martian Light Show Caused by Major Solar Storm


In [27]:
html = '<div><p>Hello, <b>world!</b></p></div>'
soup = BeautifulSoup(html, 'html.parser')
element = soup.find('div')
print(element.get_text())  # Output: 'Hello, world!'
print(element.get_text(separator='-', strip=True))  

Hello, world!
Hello,-world!


In [22]:
# Scrape the summary of the news article
news_p = slide_elem.find('div', class_='article_teaser_body').text
news_p

'For the first time in its eight years orbiting Mars, NASA’s MAVEN mission witnessed two different types of ultraviolet aurorae simultaneously, the result of solar storms that began on Aug. 27.'

In [23]:
# Quit your browsing session
browser.quit()